# 1.텍스트데이터입베딩기술 : Word2Vec, FastText

In [ ]:
#감정어 말뭉치 다운로드
!wget https://raw.githubusercontent.com/byungjooyoo/Dataset/main/corpus.txt
!pip install konlpy

--2024-05-05 22:13:07--  https://raw.githubusercontent.com/byungjooyoo/Dataset/main/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18108266 (17M) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>]  17.27M  --.-KB/s    in 0.1s    

2024-05-05 22:13:07 (178 MB/s) - ‘corpus.txt’ saved [18108266/18108266]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 24.8 MB/s eta 0:00:00


In [ ]:
# 토큰화에 8분정도 소요
from konlpy.tag import Okt
tagger = Okt()
tokened_list = []
with open('corpus.txt') as f:
  for line in f:
    token = tagger.morphs(line)
    tokened_list.append(token)

tokened_list[:4]

[['아내', '가', '드디어', '출산', '하게', '되어서', '정말', '신', '이', '나', '.', '\n'],
 ['아내', '분', '이', '출산', '을', '하시는군요', '.', '정말', '축하', '드려요', '.', '\n'],
 ['아', '지금', '정말', '신', '이', '나', '.', '\n'],
 ['잘', '된', '일이', '네', '요', '.', '\n']]

In [ ]:
from gensim.models import Word2Vec
model_1 = Word2Vec(sentences = tokened_list, window = 5, min_count = 3, workers = 4, sg = 1)
print(model_1.wv.vectors.shape)
model_1.wv.most_similar("아빠")

(23659, 100)


[('엄마', 0.9110813140869141),
 ('언니', 0.8006652593612671),
 ('누나', 0.7919774651527405),
 ('오빠', 0.7623817920684814),
 ('아버지', 0.7470435500144958),
 ('딸아이', 0.717232882976532),
 ('어머니', 0.6821330785751343),
 ('동생', 0.6766747236251831),
 ('친정엄마', 0.6681053638458252),
 ('형', 0.6644120216369629)]

In [ ]:
from gensim.models import FastText
model_2 = FastText(tokened_list, window=5, min_count=3, workers=4, sg=1)
print(model_2.wv.vectors.shape)
model_2.wv.most_similar('아빠')

(23659, 100)


[('엄마', 0.9082940816879272),
 ('누나', 0.7806795239448547),
 ('친엄마', 0.7801858186721802),
 ('언니', 0.778175413608551),
 ('오빠', 0.7577422857284546),
 ('새엄마', 0.7545873522758484),
 ('아버지', 0.7377336621284485),
 ('새어머니', 0.7209075689315796),
 ('큰아버지', 0.7205979228019714),
 ('친언니', 0.7091786861419678)]

In [ ]:
print(model_1.wv.most_similar(positive=['가정','엄마'], negative=['직장'], topn=5))
print(model_2.wv.most_similar(positive=['가정','엄마'], negative=['직장'], topn=5))

[('아빠', 0.6918531060218811), ('누나', 0.6172900795936584), ('오빠', 0.5854527950286865), ('강아지', 0.5811581015586853), ('어머니', 0.5808939337730408)]
[('아빠', 0.6976332664489746), ('어머니', 0.6084682941436768), ('강아지', 0.605941116809845), ('친엄마', 0.6051802635192871), ('누나', 0.5841014981269836)]


# 2.Subword 토큰화 도구를 이용한 단어사전 구축

## 가.Sentencepiece

In [ ]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=corpus.txt --model_prefix=MyVocab_5000 --vocab_size=5000 --model_type=bpe')
spm.SentencePieceTrainer.Train('--input=corpus.txt --model_prefix=MyVocab_100000 --vocab_size=100000 --model_type=bpe')
MyVocab1 = spm.SentencePieceProcessor()
MyVocab1.load("MyVocab_5000.model")
MyVocab2 = spm.SentencePieceProcessor()
MyVocab2.load("MyVocab_100000.model")

True

In [ ]:
test = '아내가 드디어 출산하게 되어서 정말 신이 나.  아내분이 출산을 하시는군요.'
print(test)
print(MyVocab1.encode_as_pieces(test))
print(MyVocab2.encode_as_pieces(test))
print(MyVocab1.encode_as_ids('아내'), MyVocab2.encode_as_ids('출산'))

아내가 드디어 출산하게 되어서 정말 신이 나.  아내분이 출산을 하시는군요.
['▁아내가', '▁드디어', '▁출산', '하게', '▁되어서', '▁정말', '▁신이', '▁나', '.', '▁아내분이', '▁출산', '을', '▁하시는군요', '.']
['▁아내가', '▁드디어', '▁출산하게', '▁되어서', '▁정말', '▁신이', '▁나', '.', '▁아내분이', '▁출산을', '▁하시는군요', '.']
[138] [1176]


## 나.Wordpiece

In [ ]:
!pip install tokenizers

In [ ]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer()
tokenizer.train('corpus.txt', vocab_size=5000, min_frequency=3)
tokenizer.save_model(directory='./', prefix = 'MyVocab3')
MyVocab3 = BertWordPieceTokenizer('MyVocab3-vocab.txt')

In [ ]:
text = '아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.'
print(MyVocab1.encode_as_pieces(text))
print(MyVocab3.encode(text).tokens)
print(MyVocab1.encode_as_ids('아기'))

['▁아', '기가', '▁점점', '▁', '클', '게', '▁벌써', '▁기대', '가', '▁되네', '.', '▁내가', '▁많이', '▁놀아', '줘야지', '.']
['[CLS]', '아기', '##가', '점점', '클', '##게', '벌써', '기대', '##가', '되네', '.', '내가', '많이', '놀아', '##줘야지', '.', '[SEP]']
[3966]


# 3.단어사전활용

In [ ]:
#이미 구축된 단어사전 다운로드
!wget https://github.com/byungjooyoo/Dataset/raw/main/kowiki_32000.model

--2024-05-05 22:45:41--  https://github.com/byungjooyoo/Dataset/raw/main/kowiki_32000.model
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/byungjooyoo/Dataset/main/kowiki_32000.model [following]
--2024-05-05 22:45:41--  https://raw.githubusercontent.com/byungjooyoo/Dataset/main/kowiki_32000.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 814744 (796K) [application/octet-stream]
Saving to: ‘kowiki_32000.model’

kowiki_32000.model  100%[===================>] 795.65K  --.-KB/s    in 0.03s   

2024-05-05 22:45:41 (25.8 MB/s) - ‘kowiki_32000.model’ saved [814744/814744]



In [ ]:
#말뭉치로 사용할 데이터 다운로드
!wget https://raw.githubusercontent.com/byungjooyoo/Dataset/main/corpus.txt

--2024-05-05 22:45:46--  https://raw.githubusercontent.com/byungjooyoo/Dataset/main/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18108266 (17M) [text/plain]
Saving to: ‘corpus.txt.1’

corpus.txt.1        100%[===================>]  17.27M  --.-KB/s    in 0.08s   

2024-05-05 22:45:46 (216 MB/s) - ‘corpus.txt.1’ saved [18108266/18108266]



In [ ]:
#문서들의 list로 정리 docs_list
with open('corpus.txt', 'r')  as f:
  data = f.read()
# print(data[:1000])
docs_list = data.split('""\n""')
docs_list = docs_list[:3]
docs_list

['아내가 드디어 출산하게 되어서 정말 신이 나.\n아내분이 출산을 하시는군요. 정말 축하드려요.\n아 지금 정말 신이 나.\n잘 된 일이네요.\n아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.\n좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.\n당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.\n약 종류가 많아 번거로우시겠어요.\n건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.\n현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?\n약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.\n약을 잘 챙겨 먹어 건강해지시길 바랄게요.\n고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.\n고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.\n아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.\n이 상황에서 할 수 있는 게 어떤 것이 있을까요?\n중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.\n수업을 어떤 방식으로 열심히 듣는게 좋을까요?\n재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.\n재취업 후 첫 월급이라 정말 기쁘시겠어요.\n퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 수 있어서 너무 행복해.\n이 기분을 유지하려면 어떻게 해야 할까요?\n회사생활을 열심히 해서 계속 월급을 받을거야!\n계속 원하시는 대로 회사생활이 이어지길 바랄게요.\n빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.\n기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?\n빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.\n빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.\n',
 '\n이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를 하고. 그러면 둘이 하루 종일 함께 있겠네. 생각만 해도 숨이 막혀.\n그러시군요. 은퇴 후 남편분과 함께 집에 있을 생각을 하니

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor("kowiki_32000.model")
vocab = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]
print(vocab)

['[PAD]', '[UNK]', '[BOS]', '[EOS]', '[SEP]', '[CLS]', '[MASK]', '.', '의', ',', '▁', '을', '는', ')', '에', '를', '(', '년', '은', '가', '이', '로', '과', '에서', '월', '와', '이다', '으로', '▁있다', '인', '일', '▁이', '도', '한', '▁그', '▁수', '▁1', '고', '▁2', '▁3', '하였다', '다', '하는', '▁있는', '했다', '지', '▁제', '▁4', '하고', '▁"', '"', '기', '할', '하여', '▁5', '▁《', '시', '나', '▁(', '만', "'", '해', '▁한', '▁6', "▁'", '사', '자', '▁10', '▁중', '▁한다', '-', '서', '▁8', '》', '년에', '대', '에게', '▁7', '되었다', '된', '·', '▁이후', '하게', '▁등', '▁다음', '▁9', '▁~', '▁후', '게', '▁그는', '▁되었다', '▁다른', '▁12', '▁같은', '▁때', '▁대한', '스', '리', '성', '▁및', '▁일본', '한다', '▁두', '수', '라', '적', '주', '어', '▁11', '),', '▁전', '적인', '이며', '부', '2', '▁함께', '()', '▁또한', '개', '▁위해', '면', '라는', '에는', '되어', '에서는', '차', '▁주', '히', '상', '명', '세', '화', '제', '1', ':', '▁때문에', '아', '▁formula', '▁시', '▁미국', '▁)', '진', '하기', '라고', '동', '까지', '▁그러나', '조', '들이', '군', '이나', '▁된다', '▁가장', '▁또는', '▁있으며', '트', '▁것을', '전', '산', '▁더', '장', '▁그리고', '회', '▁당시', 's', '우', '▁있었다', '미', '▁

In [ ]:
# 토큰화된 문서들의 리스트로 정리 tdocs_list
import re
tdocs_list=[]
for doc in docs_list:
    tdoc = sp.encode_as_pieces(doc)
    tdoc = [re.sub('▁', '',x) for x in tdoc]
    tdocs_list.append(' '.join(tdoc))
print(tdocs_list)

['아내 가 드 디어 출산 하게 되어서 정말 신이 나 . 아내 분이 출산 을 하 시 는 군 요 . 정말 축하 드 려 요 . 아 지금 정말 신이 나 . 잘 된 일이 네 요 . 아기 가 점점 클 게 벌써 기대 가 되 네 . 내가 많이 놀 아 줘야 지 . 좋은 아빠 가 되 실 거 같아 요 . 진심 으로 축하 드 려 요 . 당 뇨 랑 합병증 때문에 먹어 야 할 약 이 열 가지가 넘어가 니까 스트레스 야 . 약 종류 가 많아 번 거 로우 시 겠 어요 . 건강 할 때 관리 좀 잘 할 걸 하는 생각이 들 더 라고 . 현재 상황에서 변화 를 주기 위해 어떻게 하면 될 까 요 ? 약 을 잘  챙 겨 먹고 나 을 수 있도록 노력 해야 지 . 약 을 잘  챙 겨 먹어 건강 해 지 시 길 바 랄 게 요 . 고등학교 에 올라오 니 중학교 때 보다 수업 이 갑자기 어려워 져서 당황 스러워 . 고등학교 수업 이 중학교 때 와 는 다르게 갑자기 어려워 서 당황 스 러 우 시 군 요 . 아직 학 기 초 인데 내가 수업 에 잘 따라 갈 수 있을 지 걱정 돼 . 이 상황에서 할 수 있는 게 어떤 것이 있을까 요 ? 중학교 수업 을  들을 때 보다 훨 씬 더 열심히 할 거야 . 수업 을 어떤 방식으로 열심히  듣 는 게 좋 을 까 요 ? 재 취 업 이 돼 서 받게 된 첫 월급 으로 온 가족이 외 식을 할 예정 이 야 . 너무 행복 해 . 재 취 업 후 첫 월급 이라 정말 기 쁘 시 겠 어요 . 퇴직 후 다시 는 돈을 못 벌 줄 알 았 는데 이렇게 월급 으로 가족 에게 맛 있는 밥 을 살 수 있어서 너무 행복 해 . 이 기분 을 유지 하려면 어떻게 해야 할 까 요 ? 회사 생활 을 열심히 해서 계속 월급 을 받을 거 야 ! 계속 원 하시 는 대로 회사 생활 이 이어 지 길 바 랄 게 요 . 빚 을 드 디어 다  갚 게 되어서 이제 야 안 도 감이 들어 . 기분 좋 으 시 겠 어요 . 앞으로 는 어떻게 하 실 생각이 신 가 요 ? 빚 도 다  갚 았 으니 당분간 은 아무 생각 도 안 하며 살고 싶어

In [ ]:
# sklearn Countvectorizer이용 DTM 만들기
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit_transform(tdocs_list)
vocab = vect.vocabulary_
dtm = vect.transform(tdocs_list).toarray()
col = vect.get_feature_names_out()
pd.DataFrame(dtm, columns=col)

,가고,가족,가족이,가지가,감이,갑자기,같아,같이,개월,거야,...,해도,해서,해야,했는데,행복,현재,활동,회사,후에,힘든
0,0,1,1,1,1,2,1,0,0,1,...,0,1,2,0,2,1,0,2,0,0
1,1,0,0,0,0,0,0,3,1,0,...,1,0,0,0,0,0,1,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [ ]:
# sklearn을 활용한 tf-idf 변환
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(vocabulary=vocab)
tfidf = tfidfv.fit_transform(tdocs_list).toarray()
pd.DataFrame(tfidf, columns=col)

,가고,가족,가족이,가지가,감이,갑자기,같아,같이,개월,거야,...,해도,해서,해야,했는데,행복,현재,활동,회사,후에,힘든
0,0.000000,0.06257,0.06257,0.06257,0.06257,0.12514,0.047586,0.00000,0.000000,0.06257,...,0.000000,0.047586,0.12514,0.000000,0.12514,0.06257,0.000000,0.12514,0.000000,0.000000
1,0.074793,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.22438,0.074793,0.00000,...,0.074793,0.000000,0.00000,0.000000,0.00000,0.00000,0.074793,0.00000,0.074793,0.000000
2,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.075603,0.00000,0.000000,0.00000,...,0.000000,0.075603,0.00000,0.099409,0.00000,0.00000,0.000000,0.00000,0.000000,0.099409


In [ ]:
# 문서별 유사도 평가
import numpy as np
def cos_sim(A, B):
       return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))
print(round(cos_sim(dtm[0],dtm[1]),2),round(cos_sim(dtm[0],dtm[2]),2), round(cos_sim(dtm[1],dtm[2]),2))
print(round(cos_sim(tfidf[0],tfidf[1]),2),round(cos_sim(tfidf[0],tfidf[2]),2), round(cos_sim(tfidf[1],tfidf[2]),2))

0.18 0.12 0.03
0.11 0.07 0.02


In [ ]:
print(sp.encode_as_ids('가족'), sp.encode_as_ids('가족이'))

[1762] [17479]
